## 1.导入包

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import re
import numpy as np
from sklearn.model_selection import train_test_split
import time

np.set_printoptions(threshold=np.inf)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""
1.功能：
通过加载data_64x64x2.txt文件和label_a_2x64x64x2.txt文件，
将数据预处理将文本文件转换为模型要求的格式，然后继续训练和预测。
定义了一个6层卷积神经网络模型。每个卷积层后面跟着一个 ReLU 激活函数。第七层只有卷积，没有relu。
输入数据n*64*64*2,这里的一个样本64*64可以看成一个图片格式（在此次任务中是速度，两者类似）
输出是n*64*64*4
"""


'\n1.功能：\n通过加载data_64x64x2.txt文件和label_a_2x64x64x2.txt文件，\n将数据预处理将文本文件转换为模型要求的格式，然后继续训练和预测。\n定义了一个6层卷积神经网络模型。每个卷积层后面跟着一个 ReLU 激活函数。第七层只有卷积，没有relu。\n输入数据n*64*64*2,这里的一个样本64*64可以看成一个图片格式（在此次任务中是速度，两者类似）\n输出是n*64*64*4\n'

## 2. 数据预处理

In [2]:
## 从文件中读取数据并转换为三维格式
print("Start data preprocessing……")
def get_data():
    input_file = 'data/data_64x64x2.txt'
    all_data = []  # 存放所有数据
    single_file_data = []  # 存放一个文件的数据
    print(1)
    
    start_time = time.time()
    with open(input_file, 'r') as f:
        lines = f.readlines()
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"加载data用了: {execution_time} 秒！")#data用了: 292.13216066360474 秒
        
        # 匹配浮点数和科学计数法表示的浮点数的正则表达式
        pattern = r"[-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?"

        matches = re.findall(pattern, str(lines))

        result = [float(match) for match in matches]
        
        print(2)
        process_time = time.time()
        for a in range(len(result)):
            #千分之一的result输出一次
            if a%int(len(result)/1000) == int(len(result)/1000)-1:
                print("result =",str(len(result)),"a = ",str(a))
            if a == 0 or a % (4096 * 2) != 0:
                single_file_data.append(result[a])

            else:
                single_file_data = np.array(single_file_data).reshape(64, 64, 2)
                all_data.append(single_file_data)
                single_file_data = []
                single_file_data.append(result[a])
        single_file_data = np.array(single_file_data).reshape(64, 64, 2)
        all_data.append(single_file_data)
        all_data = np.array(all_data)
        print(3)
        end_time = time.time()
        process_time = end_time - start_time
        print(f"预处理data用了: {process_time} 秒！")#预处理data用了: 705.8258590698242 秒！

    print("所有样本的特征处理完毕！")
    return all_data

"""
从文件中读取数据并转换为三维列表格式
将6*2*64*64*2的数据转为12*64*64*2,再将其转为6*64*64*4
其中4是前面两个是竖着的a值，后面两个是横着的a值，用于作为标签
"""
def get_label():
    input_file = 'data/label_a_2x64x65x2.txt'
    all_label = []  # 存放所有数据
    single_file_label = []  # 存放一个文件的数据
    i = 0  # 计数，计每个文件的行个数，每4096行保存一次
    with open(input_file, 'r') as f:
        lines = f.readlines()

        # 匹配浮点数和科学计数法表示的浮点数的正则表达式
        pattern = r"[-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?"

        matches = re.findall(pattern, str(lines))

        result = [float(match) for match in matches]

        for a in range(len(result)):#716333056
            if a == 0 or a % (4096 * 2) != 0:
                single_file_label.append(result[a])

            else:
                # print(len(single_file_label))#8192
                single_file_label = np.array(single_file_label).reshape(64, 64, 2)
                all_label.append(single_file_label)
                single_file_label = []
                single_file_label.append(result[a])
        single_file_label = np.array(single_file_label).reshape(64, 64, 2)
        all_label.append(single_file_label)
        all_label = np.array(all_label)

        two_label = []
        all_4d_label = []
        num = 0
        for i in all_label:
            num += 1
            two_label.append(i)
            if num % 2 == 0:
                two_label = np.array(two_label)
                two_label = torch.tensor(two_label)
                two_label = two_label.permute(1, 2, 3, 0)
                two_label = torch.reshape(two_label, (two_label.size(0), two_label.size(1), -1))

                # 调整第三维元素的列的顺序
                column_indices = [0, 2, 1, 3]
                two_label = two_label[:, :, column_indices]

                all_4d_label.append(two_label)
                two_label = []
        all_4d_label = np.array([i.numpy() for i in all_4d_label])
        
        print("所有样本的标签处理完毕！")
        return all_4d_label
    


all_data = get_data()  # 获取所有特征
all_label = get_label()  # 获取所有标签


all_data = torch.tensor(all_data).float()
all_label = torch.tensor(all_label).float()

all_data = all_data.permute(0, 3, 1, 2)  # 调整维度顺序，将第4维插入到第2维
all_label = all_label.permute(0, 3, 1, 2)  # 调整维度顺序，将第4维插入到第2维
"""
维度转换：
(6, 64, 64, 2)
(6, 64, 64, 4)
转为
torch.Size([6, 2, 64, 64])
torch.Size([6, 4, 64, 64])
"""

print("all_data.shape:",all_data.shape)
print("all_label.shape:",all_label.shape)
# all_data.shape: torch.Size([192000, 2, 64, 64])
# all_label.shape: torch.Size([192000, 4, 64, 64])


Start data preprocessing……
1
加载data用了: 108.73742914199829 秒！
2
result = 1572864000 a =  1572863
result = 1572864000 a =  3145727
result = 1572864000 a =  4718591
result = 1572864000 a =  6291455
result = 1572864000 a =  7864319
result = 1572864000 a =  9437183
result = 1572864000 a =  11010047
result = 1572864000 a =  12582911
result = 1572864000 a =  14155775
result = 1572864000 a =  15728639
result = 1572864000 a =  17301503
result = 1572864000 a =  18874367
result = 1572864000 a =  20447231
result = 1572864000 a =  22020095
result = 1572864000 a =  23592959
result = 1572864000 a =  25165823
result = 1572864000 a =  26738687
result = 1572864000 a =  28311551
result = 1572864000 a =  29884415
result = 1572864000 a =  31457279
result = 1572864000 a =  33030143
result = 1572864000 a =  34603007
result = 1572864000 a =  36175871
result = 1572864000 a =  37748735
result = 1572864000 a =  39321599
result = 1572864000 a =  40894463
result = 1572864000 a =  42467327
result = 1572864000 a =  

result = 1572864000 a =  371195903
result = 1572864000 a =  372768767
result = 1572864000 a =  374341631
result = 1572864000 a =  375914495
result = 1572864000 a =  377487359
result = 1572864000 a =  379060223
result = 1572864000 a =  380633087
result = 1572864000 a =  382205951
result = 1572864000 a =  383778815
result = 1572864000 a =  385351679
result = 1572864000 a =  386924543
result = 1572864000 a =  388497407
result = 1572864000 a =  390070271
result = 1572864000 a =  391643135
result = 1572864000 a =  393215999
result = 1572864000 a =  394788863
result = 1572864000 a =  396361727
result = 1572864000 a =  397934591
result = 1572864000 a =  399507455
result = 1572864000 a =  401080319
result = 1572864000 a =  402653183
result = 1572864000 a =  404226047
result = 1572864000 a =  405798911
result = 1572864000 a =  407371775
result = 1572864000 a =  408944639
result = 1572864000 a =  410517503
result = 1572864000 a =  412090367
result = 1572864000 a =  413663231
result = 1572864000 

result = 1572864000 a =  740818943
result = 1572864000 a =  742391807
result = 1572864000 a =  743964671
result = 1572864000 a =  745537535
result = 1572864000 a =  747110399
result = 1572864000 a =  748683263
result = 1572864000 a =  750256127
result = 1572864000 a =  751828991
result = 1572864000 a =  753401855
result = 1572864000 a =  754974719
result = 1572864000 a =  756547583
result = 1572864000 a =  758120447
result = 1572864000 a =  759693311
result = 1572864000 a =  761266175
result = 1572864000 a =  762839039
result = 1572864000 a =  764411903
result = 1572864000 a =  765984767
result = 1572864000 a =  767557631
result = 1572864000 a =  769130495
result = 1572864000 a =  770703359
result = 1572864000 a =  772276223
result = 1572864000 a =  773849087
result = 1572864000 a =  775421951
result = 1572864000 a =  776994815
result = 1572864000 a =  778567679
result = 1572864000 a =  780140543
result = 1572864000 a =  781713407
result = 1572864000 a =  783286271
result = 1572864000 

result = 1572864000 a =  1107296255
result = 1572864000 a =  1108869119
result = 1572864000 a =  1110441983
result = 1572864000 a =  1112014847
result = 1572864000 a =  1113587711
result = 1572864000 a =  1115160575
result = 1572864000 a =  1116733439
result = 1572864000 a =  1118306303
result = 1572864000 a =  1119879167
result = 1572864000 a =  1121452031
result = 1572864000 a =  1123024895
result = 1572864000 a =  1124597759
result = 1572864000 a =  1126170623
result = 1572864000 a =  1127743487
result = 1572864000 a =  1129316351
result = 1572864000 a =  1130889215
result = 1572864000 a =  1132462079
result = 1572864000 a =  1134034943
result = 1572864000 a =  1135607807
result = 1572864000 a =  1137180671
result = 1572864000 a =  1138753535
result = 1572864000 a =  1140326399
result = 1572864000 a =  1141899263
result = 1572864000 a =  1143472127
result = 1572864000 a =  1145044991
result = 1572864000 a =  1146617855
result = 1572864000 a =  1148190719
result = 1572864000 a =  114

result = 1572864000 a =  1465909247
result = 1572864000 a =  1467482111
result = 1572864000 a =  1469054975
result = 1572864000 a =  1470627839
result = 1572864000 a =  1472200703
result = 1572864000 a =  1473773567
result = 1572864000 a =  1475346431
result = 1572864000 a =  1476919295
result = 1572864000 a =  1478492159
result = 1572864000 a =  1480065023
result = 1572864000 a =  1481637887
result = 1572864000 a =  1483210751
result = 1572864000 a =  1484783615
result = 1572864000 a =  1486356479
result = 1572864000 a =  1487929343
result = 1572864000 a =  1489502207
result = 1572864000 a =  1491075071
result = 1572864000 a =  1492647935
result = 1572864000 a =  1494220799
result = 1572864000 a =  1495793663
result = 1572864000 a =  1497366527
result = 1572864000 a =  1498939391
result = 1572864000 a =  1500512255
result = 1572864000 a =  1502085119
result = 1572864000 a =  1503657983
result = 1572864000 a =  1505230847
result = 1572864000 a =  1506803711
result = 1572864000 a =  150

'\n维度转换：\n(6, 64, 64, 2)\n(6, 64, 64, 4)\n转为\ntorch.Size([6, 2, 64, 64])\ntorch.Size([6, 4, 64, 64])\n'

In [6]:
# 将数据写入文件，格式只能为二进制
np.save('./data/all_data.npy', all_data)
#直接加载npy文件为numpy格式
# all_data = np.load('./data/all_data.npy')

# # 将数据写入文件，格式只能为二进制
np.save('./data/all_label.npy', all_label)
# #直接加载npy文件为numpy格式
# all_label = np.load('./data/all_label.npy')
print("all data has been saved and the format is numpy!")

all data has been saved and the format is numpy!


In [ ]:
# 直接加载npy文件为numpy格式
#当数据预处理的代码运行完保存处理后的数据后，可直接运行以下代码加载数据
# all_data = np.load('./data/all_data.npy')
# all_label = np.load('./data/all_label.npy')

# all_data = torch.tensor(all_data).float()
# all_label = torch.tensor(all_label).float()

## 3.CNN

In [7]:

#神经网络模型
class Net(nn.Module):
    def __init__(self, num_output_channels):
        super(Net, self).__init__()

        # 第一层卷积，使用64个大小为3x3的卷积核，输入数据的shape为2x64x64，使用ReLU激活函数。
        # 输入通道数为2，输出通道数恒为64
        self.conv1 = nn.Conv2d(2, 64, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()

        # 第二层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()

        # 第三层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()

        # 第四层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()

        # 第五层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数
        self.conv5 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu5 = nn.ReLU()

        # 第六层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数
        self.conv6 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu6 = nn.ReLU()

        # 第七层卷积，使用64个大小为3x3的卷积核，使用ReLU激活函数,输出数据为4*64*64
        # 输入通道数为64，输出通道数恒为4
        self.conv7 = nn.Conv2d(64, num_output_channels, kernel_size=3, stride=1, padding=1)



    def forward(self, x):
        x = self.conv1(x)  # x:torch.Size([10, 2, 64, 64])
        x = self.relu1(x)

        x = self.conv2(x)  # x:torch.Size([10, 64, 64, 64])
        x = self.relu2(x)

        x = self.conv3(x)  # x:  torch.Size([10, 64, 64, 64])
        x = self.relu3(x)

        x = self.conv4(x)
        x = self.relu4(x)

        x = self.conv5(x)
        x = self.relu5(x)

        x = self.conv6(x)
        x = self.relu6(x)

        x = self.conv7(x)

        # 输出数据shape: torch.Size([10, 4, 64, 64]),10是batch_size大小
        return x



## 4.训练和测试模型

In [8]:
def train(num_output_channels):

    # 定义一个batch包含的样本数目
    batch_size =20

    # 生成数据集
    x_train, x_test, y_train, y_test = train_test_split(all_data, all_label, test_size=0.2)


    # 划分数据集
    trainset = torch.utils.data.TensorDataset(x_train, y_train)
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=True)

    # 划分数据集
    testset = torch.utils.data.TensorDataset(x_test, y_test)
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=batch_size, shuffle=True)


    # 创建模型实例，并将模型移动到GPU设备上进行计算
    net = Net(num_output_channels).to(device)
    
    # 加速训练：如果有多个GPU，则使用DataParallel模块
    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
        
    # 定义损失函数为均方误差
    criterion = nn.MSELoss()  # 将预测值与真实标签之间的差值求平方和，再除以样本数n来得到平均损失值

    # 定义优化器为Adam优化器
    optimizer = optim.Adam(net.parameters())

    print('begin to train!!!')

    # 训练模型
    num_epochs = 2  # 训练轮数
    for epoch in range(num_epochs):
        running_train_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)

            # 将参数的梯度设为0
            optimizer.zero_grad()

            # 前向传播+后向传播+优化
            outputs = net(inputs)
            # inputs:torch.Size([10, 2, 64, 64])
            # outputs:torch.Size([10, 4, 64, 64])
            """
            criterion：表示所选的损失函数，这里选择的是MSE均方误差
            outputs：模型在输入后得到的输出，它的形状是(batch_size, num_output_channels, height, width)
            """
            loss = criterion(outputs, labels)  # labels   :torch.Size([10, 120, 64, 64])
            loss.backward()  # 有了损失值后，就可以根据反向传播算法来更新模型参数，使得预测值更接近真实标签。
            optimizer.step()

            # 统计损失值
            running_train_loss += loss.item()  # 在训练过程中，目标就是通过反向传播和优化算法尽可能地减小该损失值，提高模型的性能。

            # 每20个batch打印一次平均损失值，这里batch-size=10,相当于每100个样本打印一次loss
            if i % 20 == 19:
#             if i % 1 == 0:
                print('[epoch:  %d, batch:%5d] train loss: %.3f' %
                      (epoch + 1, i + 1, train_loss))
                running_train_loss = 0.0


        running_test_loss = 0.0
        for i, (inputs, labels) in enumerate(testloader):
            inputs, labels = inputs.to(device), labels.to(device)
            # 前向传播(预测)+后向传播+优化
            outputs = net(inputs)
            # inputs:torch.Size([10, 2, 64, 64])
            # outputs:torch.Size([10, 4, 64, 64])
            """
            criterion：表示所选的损失函数，这里选择的是MSE均方误差
            outputs：模型在输入后得到的输出，它的形状是(batch_size, num_output_channels, height, width)
            """
            loss = criterion(outputs, labels)  # labels   :torch.Size([10, 120, 64, 64])

            # 统计损失值
            running_test_loss += loss.item()  # 在训练过程中，目标就是通过反向传播和优化算法尽可能地减小该损失值，提高模型的性能。

            # 每20个batch打印一次平均损失值，这里batch-size=10,相当于每100个样本打印一次loss
            if i % 20 == 19:
#             if i % 1 == 0:
                print('[epoch:  %d, batch:%5d] test loss: %.3f' %
                      (epoch + 1, i + 1, test_loss))
                running_test_loss = 0.0


    print('all of tasks Finished')


train(num_output_channels=4)


begin to train!!!


NameError: name 'train_loss' is not defined